In [ ]:
let parseLine (s:string) = 
  let [|patterns; output|] = 
    s.Split('|') 
    |> Array.map (fun (sec:string) -> sec.Split(' ', StringSplitOptions.RemoveEmptyEntries) |> Array.map Set.ofSeq)
  (patterns |> Set.ofArray, output)

let input = File.ReadAllLines "input.txt" |> Seq.map parseLine

input
|> Seq.collect snd
|> Seq.filter (Set.count >> (function 2 | 3 | 4 | 7 -> true | _ -> false))
|> Seq.length

321

In [ ]:
let properWiring =
  seq [
    set ['a';'b';'c';    'e';'f';'g'];
    set [        'c';        'f';   ];
    set ['a';    'c';'d';'e';    'g'];
    set ['a';    'c';'d';    'f';'g'];
    set [    'b';'c';'d';    'f';   ];
    set ['a';'b';    'd';    'f';'g'];
    set ['a';'b';    'd';'e';'f';'g'];
    set ['a';    'c';        'f';   ];
    set ['a';'b';'c';'d';'e';'f';'g'];
    set ['a';'b';'c';'d';    'f';'g']
  ]

let countChars = Seq.collect id >> Seq.countBy id 
let properCharCounts = countChars properWiring |> Map.ofSeq
let properCharCount c = Map.find c properCharCounts


type Rewiring = (char * char) list

let rewire (swaps: Rewiring) pattern = 
  let swap pattern (c1, c2) =
    match Set.contains c1 pattern, Set.contains c2 pattern with
    | true, true | false, false -> pattern
    | false, true -> pattern |> Set.add c1 |> Set.remove c2
    | true, false -> pattern |> Set.remove c1 |> Set.add c2
  swaps |> Seq.rev |> Seq.fold swap pattern
  
let tryFixWiring (patterns: char Set Set) =
  let rec tryWiring mapping = function
  | c :: rest ->
    let p = patterns |> Set.map (rewire mapping)
    if set properWiring = p then Some mapping else
      tryWiring mapping rest
      |> Option.orElse
        (countChars p 
        |> Seq.filter (fun (c2, i) -> List.contains c2 rest && (properCharCount c) = i)
        |> Seq.map fst
        |> Seq.tryPick (fun c2 -> tryWiring ((c, c2) :: mapping) rest))
  | _ -> None
  
  tryWiring [] (patterns |> Set.unionMany |> Set.toList)
  |> Option.defaultWith (fun () -> failwith "Successful rewiring not found")

let decode (patterns, output) = 
  let rewiring = tryFixWiring patterns 
  
  output
  |> Array.map (rewire rewiring) 
  |> Array.map (fun x -> Seq.findIndex ((=) x) properWiring)
  |> Array.fold (fun acc n -> acc * 10 + n) 0

input |> Seq.sumBy decode

map [('a', 8); ('b', 6); ('c', 8); ('d', 7); ('e', 4); ('f', 9); ('g', 7)]


1028926